In [ ]:
import os
import kagglehub
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
dataset_path = kagglehub.dataset_download('kssanjaynithish03/retinal-fundus-images')
print(f"Dataset downloaded to: {dataset_path}")

100%|██████████| 2.32G/2.32G [01:52<00:00, 22.1MB/s]

Extracting files...


Dataset downloaded to: /root/.cache/kagglehub/datasets/kssanjaynithish03/retinal-fundus-images/versions/3


In [ ]:
base_path = os.path.dirname(dataset_path)
print(f"\nBase path contents: {os.listdir(base_path)}")


Base path contents: ['3']


In [ ]:
def find_data_dirs(root_path):
    for root, dirs, files in os.walk(root_path):
        if 'train' in dirs and 'test' in dirs:
            return os.path.join(root, 'train'), os.path.join(root, 'test')
        elif 'training' in dirs and 'testing' in dirs:
            return os.path.join(root, 'training'), os.path.join(root, 'testing')
    return None, None

train_dir, test_dir = find_data_dirs(base_path)

if not train_dir or not test_dir:
    for root, dirs, files in os.walk(base_path):
        if 'train' in root.lower() and os.path.isdir(root):
            train_dir = root
            test_dir = root.replace('train', 'test')
            if os.path.exists(test_dir):
                break

In [ ]:
if not os.path.exists(train_dir):
    raise FileNotFoundError(f"Train directory not found. Searched in: {base_path}")
if not os.path.exists(test_dir):
    raise FileNotFoundError(f"Test directory not found. Searched in: {base_path}")

print(f"\nFound train directory: {train_dir}")
print(f"Found test directory: {test_dir}")



Found train directory: /root/.cache/kagglehub/datasets/kssanjaynithish03/retinal-fundus-images/versions/3/Retinal Fundus Images/train
Found test directory: /root/.cache/kagglehub/datasets/kssanjaynithish03/retinal-fundus-images/versions/3/Retinal Fundus Images/test


In [ ]:
batch_size = 64
image_size = (256, 256)

train_dataset = image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="categorical",
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="training"
)

test_dataset = image_dataset_from_directory(
    test_dir,
    labels="inferred",
    label_mode="categorical",
    batch_size=batch_size,
    image_size=image_size,
    shuffle=False
)

print("\nSuccessfully loaded datasets!")
print(f"Train batches: {len(train_dataset)}")
print(f"Test batches: {len(test_dataset)}")

Found 20077 files belonging to 11 classes.
Using 16062 files for training.
Found 1236 files belonging to 11 classes.

Successfully loaded datasets!
Train batches: 251
Test batches: 20


In [ ]:
class_names = train_dataset.class_names
print("Class names:", class_names)
print("\nTraining dataset:")
for images, labels in train_dataset.take(1):
    print("Images shape:",images.shape)
    print("Labels shape:",labels.shape)
print("\nTest dataset:")
for images, labels in test_dataset.take(1):
    print("Images shape:",images.shape)
    print("Labels shape:",labels.shape)

Class names: ['1.Dry AMD', '10.Glaucoma', '11.Normal Fundus', '2.Wet AMD', '3.Mild DR', '4.Moderate DR', '5.Severe DR', '6.Proliferate DR', '7.Cataract', '8.Hypertensive Retinopathy', '9.Pathological Myopia']

Training dataset:
Images shape: (64, 256, 256, 3)
Labels shape: (64, 11)

Test dataset:
Images shape: (64, 256, 256, 3)
Labels shape: (64, 11)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
model=Sequential([
    Conv2D(32,(3, 3),activation='relu',padding='same',input_shape=(256, 256, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64,(3, 3),activation='relu',padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(128,(3, 3),activation='relu',padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(256,(3, 3),activation='relu',padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(512,(3, 3),activation='relu',padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(512,(3, 3),activation='relu',padding='same'),
    MaxPooling2D((2, 2)),
    Flatten(),

    Dense(1024,activation='relu'),
    Dense(512,activation='relu'),
    Dense(256,activation='relu'),
    Dense(128,activation='relu'),

    Dense(len(class_names),activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 256, 256, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 128, 128, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 128, 128, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 64, 64, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 32, 32, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 16, 16, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 16, 16, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 8, 8, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 8, 8, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 4, 4, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       8,389,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 11)                  │           1,419 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,008,459 (49.62 MB)

 Trainable params: 13,008,459 (49.62 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

epochs = 50
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs,
    callbacks=[early_stopping]
)

Epoch 1/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 96s 300ms/step - accuracy: 0.3375 - loss: 1.8127 - val_accuracy: 0.5558 - val_loss: 1.0202
Epoch 2/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 58s 229ms/step - accuracy: 0.6183 - loss: 0.9081 - val_accuracy: 0.6675 - val_loss: 0.8203
Epoch 3/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 81s 225ms/step - accuracy: 0.6690 - loss: 0.7635 - val_accuracy: 0.6748 - val_loss: 0.7696
Epoch 4/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 81s 223ms/step - accuracy: 0.6991 - loss: 0.6776 - val_accuracy: 0.6990 - val_loss: 0.6939
Epoch 5/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 57s 226ms/step - accuracy: 0.7261 - loss: 0.6220 - val_accuracy: 0.6917 - val_loss: 0.7751
Epoch 6/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 57s 228ms/step - accuracy: 0.7448 - loss: 0.5720 - val_accuracy: 0.7063 - val_loss: 0.7138
Epoch 7/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 80s 220ms/step - accuracy: 0.7648 - loss: 0.5236 - val_accuracy: 0.7249 - val_loss: 0.7203
Epoch 8/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 85s 233ms/step - accuracy: 0.7745 - loss: 0

In [ ]:
test_loss, test_accuracy=model.evaluate(test_dataset)
print(f"Test Accuracy: {test_accuracy:.2f}")

20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 220ms/step - accuracy: 0.7392 - loss: 0.6218
Test Accuracy: 0.73


In [ ]:
from google.colab import files
model.save('retinal_model.h5')
files.download('retinal_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>